# Lab 01. LeNet-5 실습 1
---

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

c:\Users\kss34\anaconda3\envs\pytorchPractice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 데이터 다운로드 (CIFAR-10)

In [2]:
# Transform
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, download=False, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(root="./data", train=False, download=False, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

## LeNet-5 모델

In [4]:
class LeNet(nn.Module) : 
    def __init__(self): 
        super(LeNet, self).__init__() 
        self.conv1 = nn.Conv2d(3, 32, 3)    # 입력 채널, 출력 채널 수, 커널 사이즈 
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, 3)    # 입력 채널, 출력 채널 수, 커널 사이즈 
        self.fc1 = nn.Linear(64 * 6 * 6 , 64)    # 크기 조정
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x) : 
        x = self.pool(torch.relu(self.conv1(x)))    # feature map >> 14 x 14 
        x = self.pool(torch.relu(self.conv2(x)))    # feature map >> 6 x 6 
        x = torch.flatten(x, 1)    # 1차원으로 펄치기   # 64 x 6 x 6, 64 
        x = torch.relu(self.fc1(x))    # 64 , 10
        x = self.fc2(x)
        
        return x 

## 모델 학습 및 평가 함수 구현 

In [5]:
def train_and_eval(model) : 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # 학습 Loop
    for epoch in range(5) : 
        runing_loss = 0.0 
        for i, data in enumerate(train_loader, 0) : 
            images, labels = data
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            runing_loss += loss.item()
            """
            매 200번째 미니 배치마다 조건이 참이 되어 손실이 출력되고 runing_loss가 재설정됩니다. 
            이는 미니 배치가 190번째일 때마다 출력 및 초기화가 수행됩니다.
            """
            if i % 200 == 190 : 
                print('[%d, %5d] loss : %.3f' % (epoch + 1, i + 1, runing_loss / 200))
                runing_loss = 0.0
                
    print("Finshed Training ....")
    
    
    # 모델 평가 
    correct = 0
    total = 0 
    with torch.no_grad():
        for data in test_loader : 
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = 100 * correct / total
    print("Acc >> %.2f %%"% acc)

## 실행

In [6]:
print("LeNet-5")
model = LeNet()
train_and_eval(model)

# 파라미터 수 
print("LeNet", sum(p.numel() for p in model.parameters()))

LeNet-5
[1,   191] loss : 2.154
[1,   391] loss : 2.084
[1,   591] loss : 1.953
[2,   191] loss : 1.680
[2,   391] loss : 1.677
[2,   591] loss : 1.638
[3,   191] loss : 1.485
[3,   391] loss : 1.539
[3,   591] loss : 1.505
[4,   191] loss : 1.398
[4,   391] loss : 1.434
[4,   591] loss : 1.423
[5,   191] loss : 1.324
[5,   391] loss : 1.351
[5,   591] loss : 1.350
Finshed Training ....
Acc >> 55.66 %
LeNet 167562
